In [1]:
import src.view.pnl_view as pnlview
from multiprocessing import Pool, Process, Manager, cpu_count
from src.core.logger import Logger
from src.core.stopwatch import Stopwatch
from src.core.stopwatch_logger import StopwatchLogger
from src.synthetic_book_merger import SyntheticBookMerger
from src.synthetic_security_loader import SyntheticSecurityLoader
import src.sniper_opp_loader
from src.sniper_opp_loader import SniperOppLoader

from src.view import data_view as dview
from src.view.opp_view import OppView, to_opp_view, display_opp_view
from src.view.cell_view import CellView
from src.view.color import Color, Colors, WebColors
from src.pcap_location_params import PCapLocationParams
from src.core.postgres_connection import PostgresConnection
from src.view.cell_view import CellView
from src.market_date import *

import pandas as pd
import numpy as np
import sqlite3
import time
import matplotlib.pyplot as plt
import os.path

# pandarallel.initialize(progress_bar=True, use_memory_fs=True)
pd.options.display.width = 920
pd.options.display.max_columns = 500
pd.options.display.max_rows = 220
pd.options.display.max_rows = 263

In [2]:
import importlib
import src.view.opp_view
import src.view.data_view as dview
import src.core.sort_utils as sort_utils
import src.core.numpy_utils as np_utils
import src.view.edge_summary_view
import src.view.opp_dur_count_view
import src.synthetic_security_loader
import src.synthetic_book_merger
import src.sniper_opp_loader
import src.sniper_opp_dur_summarizer
import src.view.opp_dur_view
importlib.reload(sort_utils)
importlib.reload(module=src.view.opp_view)
importlib.reload(module=dview)
importlib.reload(module=src.view.edge_summary_view)
importlib.reload(module=src.synthetic_security_loader)
importlib.reload(module=src.synthetic_book_merger)
importlib.reload(module=src.sniper_opp_loader)
importlib.reload(module=src.sniper_opp_dur_summarizer)
importlib.reload(module=src.view.opp_dur_view)
importlib.reload(module=src.view.opp_dur_count_view)
from src.view.opp_view import OppView
from src.view.edge_summary_view import get_edge_summary
from src.synthetic_security_loader import SyntheticSecurityLoader
from src.synthetic_book_merger import SyntheticBookMerger
from src.view.opp_dur_view import OppDurView

In [125]:
synth_sec = SyntheticSecurityLoader().result()
synth_sec

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], dtype='int64', name='sid')
RangeIndex(start=0, stop=39, step=1)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 1 to 39
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                39 non-null     int64  
 1   R106J_MEMBER      39 non-null     float64
 2   R106J_NON_MEMBER  39 non-null     float64
 3   IIP_MEMBER        39 non-null     float64
 4   IIP_NON_MEMBER    39 non-null     float64
 5   CBIP_MEMBER       39 non-null     float64
 6   CBIP_NON_MEMBER   39 non-null     float64
 7   NON_MEMBER        39 non-null     float64
dtypes: float64(7), int64(1)
memory usage: 2.7 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------

,id,symbol,R106J_MEMBER,R106J_NON_MEMBER,IIP_MEMBER,IIP_NON_MEMBER,CBIP_MEMBER,CBIP_NON_MEMBER,NON_MEMBER
0,1,+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),1.11,2.88,6.18,6.18,11.52,11.52,11.52
1,2,+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...,1.48,3.84,8.24,8.24,15.36,15.36,15.36
2,3,+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-...,1.48,3.84,8.24,8.24,15.36,15.36,15.36
3,4,+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,1.20,2.94,6.24,6.24,11.02,11.02,11.58
4,5,+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-...,1.48,3.84,8.24,8.24,15.36,15.36,15.36
5,6,+ZSQ1 -(ZSQ1-ZSU1) -ZSU1,0.83,1.98,4.18,4.18,7.18,7.18,7.74
6,7,+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,0.83,1.98,4.18,4.18,7.18,7.18,7.74
7,8,+ZSN2 -(ZSN2-ZSX2) -ZSX2,0.83,1.98,4.18,4.18,7.18,7.18,7.74
8,9,+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),1.11,2.88,6.18,6.18,11.52,11.52,11.52
9,10,+ZSN1 -(ZSN1-ZSX1) -ZSX1,0.83,1.98,4.18,4.18,7.18,7.18,7.74


In [4]:
synth_sec = SyntheticSecurityLoader().result()
market_dates = get_feather_dates(340, name = f'security', month=[7])
opps = src.sniper_opp_loader.from_postgres(host='titan')

topps = opps.copy()
topps['sym'] = topps.index.get_level_values('symbol').astype('str')
topps = topps.loc[~(topps['sym'].str.match(src.sniper_opp_loader.uncommon_poly_regex()))]
topps_sym_groupby = topps.groupby('symbol')
topps_dur_summ = src.sniper_opp_dur_summarizer.get_opp_dur_summary(topps_sym_groupby)

topps_good = topps.loc[(topps['entry_ticks'] > 0) & (topps['lsn_win'] > pd.Timedelta('55us'))]
topps_good_sym_groupby = topps_good.groupby('symbol')
topps_good_dur_summ = src.sniper_opp_dur_summarizer.get_opp_dur_summary(topps_good_sym_groupby)

opps_sdict = {src.sniper_opp_dur_summarizer.fmt_quantile_col(.1 * q, prefix='lsn_win'): lambda x: int(dview.TimedeltaView(x).normal_microseconds()) for q in range(1, 10)}

zs_opps = topps.loc[(topps['entry_ticks'] > 0)].copy()
zs_opps  = zs_opps.loc[(zs_opps['sym'].str.match('^\+\({,1}ZS.*'))]

dur_threshes=[pd.Timedelta(t, unit='us') for t in [55, 75, 100, 150, 200]]
zs_opp_threshes = src.sniper_opp_dur_summarizer.get_opp_dur_summary(zs_opps[['sym', 'lsn_win']].groupby('sym'), durs='lsn_win', threshes=dur_threshes)
zs_opp_threshes = zs_opp_threshes.set_index(zs_opp_threshes.index.get_level_values('sym'))

In [261]:
secs = pd.read_feather(PCapLocationParams(340).feather_file_path(name='security', date='2021-05-26'))
secs = secs.loc[secs['asset'] == 'ZS']
secs.loc[secs['symbol'].isin(['ZSQ1-ZSU1', 'ZSQ1-ZSX1', 'ZSU1-ZSX1', 'ZSQ1-ZSF2', 'ZSU1-ZSF2', 'ZSX1-ZSF2'])][['symbol']]
# secs['sym'] = secs['symbol'].astype('str')
# secs = secs.loc[secs['sym'].str.match('')]

,symbol
248,ZSX1-ZSF2
1721,ZSQ1-ZSX1
2041,ZSU1-ZSF2
2338,ZSQ1-ZSF2
2667,ZSU1-ZSX1
3238,ZSQ1-ZSU1


In [23]:
# zs_opps
threshes={
    0: None,
    2: CellView(bg='skyblue'),
    4: CellView(bg='gold'),
    8: CellView(bg='palegreen')
}
zs_opp_threshes = zs_opp_threshes.sort_values('lsn_win>=55us', ascending=False).head(60).copy()
# zs_opp_threshes = zs_opp_threshes.sort_values('lsn_win>=55us', ascending=False).head(20)
v = src.view.opp_dur_count_view.to_opp_dur_view(zs_opp_threshes, opp_durs=threshes, dur_type=int)
src.view.opp_dur_count_view.display_opp_dur_view(v, dur_cols=[f'lsn_win>={int(d.value / 1000)}us' for d in dur_threshes])

,n_opps,lsn_win>=55us,lsn_win>=75us,lsn_win>=100us,lsn_win>=150us,lsn_win>=200us
sym,,,,,,
+ZSU1 -(ZSU1-ZSX1) -(ZSX1-ZSH2) -ZSH2,60,33,33,33,33,33
+(ZSQ1-ZSU1) -(ZSQ1-ZSX1) +(ZSU1-ZSX1),116,33,32,31,27,25
+ZSF2 -(ZSF2-ZSH2) -(ZSH2-ZSK2) -ZSK2,53,32,32,31,30,30
+ZSQ1 -ZSU1 -(ZSQ1-ZSX1) +(ZSU1-ZSX1),54,26,25,24,23,23
+(ZSQ1-ZSU1) +ZSU1 -(ZSQ1-ZSF2) -ZSF2,34,26,26,25,25,24
+ZSU1 -(ZSU1-ZSX1) -ZSX1,52,26,26,25,25,24
+ZSX1 -(ZSX1-ZSF2) -(ZSF2-ZSH2) -ZSH2,55,23,21,19,18,17
+ZSQ1 -(ZSQ1-ZSX1) -ZSX1,39,23,23,23,23,22
+ZSU1 -(ZSU1-ZSH2) -ZSH2,37,23,23,22,22,22


In [12]:
tmp = src.sniper_opp_dur_summarizer.get_opp_dur_summary(zs_opps.groupby('sym'))
tmp = tmp.set_index(tmp.index.get_level_values('sym'))
def get_dur_selector(dur):
    return np.where(dur > pd.Timedelta('0s'), 'pos-dur', 'neg-dur')

def display_opp_durs(df, **kwargs):
    header_style = kwargs.get('header_style') or CellView(bg=Colors.header_background(), border='solid', bdc='white', valign='top')
    even_style = kwargs.get('even') or kwargs.get('even_style') or CellView(fg=Colors.black(), bg=Color(171,171,171))
    odd_style = kwargs.get('odd') or kwargs.get('odd_style') or CellView(fg=Colors.black(), bg=Color(196,196,196))
    hover_style = kwargs.get('hover') or kwargs.get('hover_style') or CellView(bg='slateblue')
    opp_origin_style = kwargs.get('opp_origin_style') or CellView(bdts='solid', bdtc='CornflowerBlue')
    opp_origin_selector = kwargs.get('opp_origin_selector') or 'opp-origin'

    def get_selectors(df):
        qcols = [src.sniper_opp_loader.fmt_quantile_col(.1 * q, prefix='lsn_win') for q in range(1, 10)]
        return pd.DataFrame(
            data=dict(
                [('n_opps', np.empty(len(df)))] +
                [(c, df[c].apply(get_dur_selector)) for c in qcols]))

    df = df.drop(columns=[src.sniper_opp_dur_summarizer.fmt_quantile_col(.1 * q, prefix='fsn_win') for q in range(1, 10)])\
        .sort_values('n_opps', ascending=False)
    odv = src.view.opp_dur_view.to_opp_dur_view(df)
    return src.view.opp_dur_view.display_opp_dur_view(odv)

display_opp_durs(tmp.reset_index())

,sym,n_opps,lsn_win_10,lsn_win_20,lsn_win_30,lsn_win_40,lsn_win_50,lsn_win_60,lsn_win_70,lsn_win_80,lsn_win_90
88,+(ZSQ1-ZSU1) -(ZSQ1-ZSX1) +(ZSU1-ZSX1),116,-14391,-6270,-3189,-1314,-604,-131,35,352,2827
73,+(ZSQ1-ZSU1) +ZSU1 -(ZSQ1-ZSX1) -ZSX1,63,-8434,-5331,-3141,-1548,-116,33,625,29985,3077626
264,+ZSU1 -(ZSU1-ZSX1) -(ZSX1-ZSH2) -ZSH2,60,-9853,-3475,-49,42,797,11882,92144,627558,2341369
236,+ZSQ1 -(ZSQ1-ZSU1) -(ZSU1-ZSX1) -ZSX1,57,-31434,-8080,-4868,-3068,-1182,-533,26,231,4530
272,+ZSX1 -(ZSX1-ZSF2) -(ZSF2-ZSH2) -ZSH2,55,-12588,-5929,-1823,-200,7,66,244,941,9518
246,+ZSQ1 -ZSU1 -(ZSQ1-ZSX1) +(ZSU1-ZSX1),54,-18212,-8439,-4753,-0,48,279,12394,528069,1936286
172,+ZSF2 -(ZSF2-ZSH2) -(ZSH2-ZSK2) -ZSK2,53,-7255,-1851,-168,69,487,1978,7569,25752,61105
268,+ZSU1 -(ZSU1-ZSX1) -ZSX1,52,-8444,-4007,-978,-1,66,1553,14848,58865,239287
94,+(ZSQ1-ZSX1) -(ZSQ1-ZSF2) +(ZSX1-ZSF2),51,-19741,-8603,-5242,-1520,-567,9,152,1534,8130
174,+ZSF2 -(ZSF2-ZSH2) -ZSH2,44,-6602,-507,-125,-42,67,172,659,1001,9723


In [62]:
tmp = topps
tmp = tmp.loc['2021-07-18':,:]
tmp = tmp.loc[(tmp['is_direct']) & (tmp['entry_ticks'] > 0)].copy()
tmp['sym'] = tmp.index.get_level_values('symbol').astype('str')
tmp = tmp.loc[~tmp['sym'].str.match('^(\+[0-9]|.).*:.*')]
sdict = {f'lsn_win_{10 * q:.0f}': lambda x: int(dview.TimedeltaView(x).normal_microseconds()) for q in range(1, 10)}
tmp = src.sniper_opp_dur_summarizer.get_opp_dur_summary(tmp.groupby('sym'))
tmp.loc[(tmp['lsn_win_50'] > pd.Timedelta('0us'))]\
    .drop(columns=[f'fsn_win_{10 * q:.0f}' for q in range(1, 10)])\
    .sort_values('n_opps', ascending=False)\
    .head(25).style.format(sdict)

,,n_opps,lsn_win_10,lsn_win_20,lsn_win_30,lsn_win_40,lsn_win_50,lsn_win_60,lsn_win_70,lsn_win_80,lsn_win_90
sym,,,,,,,,,,,
+(ZWZ1-ZWH2) -(ZWZ1-ZWN2) +(ZWH2-ZWN2),0,69,-975,41,47,69,80,104,118,134,228
+(ZWZ1-ZWK2) -(ZWZ1-ZWN2) -(ZWH2-ZWK2) +(ZWH2-ZWN2),0,32,-8797,-1853,38,64,75,79,113,136,339
+(ZWU1-ZWZ1) -(ZWU1-ZWH2) +(ZWZ1-ZWN2) -(ZWH2-ZWN2),0,30,-9997,-2324,7,61,73,78,99,127,198
+ZWU1 -(ZWU1-ZWH2) -(ZWH2-ZWN2) -ZWN2,0,17,-82970,-51644,-11545,-4057,40,41,42,47,64
+(ZMQ1-ZMZ1) -(ZMQ1-ZMN2) +(ZMZ1-ZMN2),0,15,27,29,34,44,52,87,106,185,250
+(ZMQ1-ZMU1) +(ZMU1-ZMZ1) -(ZMQ1-ZMN2) +(ZMZ1-ZMN2),0,13,26,29,31,36,52,102,124,170,270
+(ZSQ1-ZSU1) -(ZSQ1-ZSF2) +(ZSU1-ZSF2),0,12,-5906,-4517,-1607,-70,24,56,67,91,118
+(ZMQ1-ZMV1) +(ZMV1-ZMZ1) -(ZMQ1-ZMN2) +(ZMZ1-ZMN2),0,12,28,32,48,53,66,91,105,173,281
+(ZMQ1-ZMZ1) -(ZMQ1-ZMN2) +(ZMZ1-ZMK2) +(ZMK2-ZMN2),0,11,28,49,52,74,79,91,100,107,1402


In [140]:
tmp = topps
tmp = tmp.loc[(tmp['is_direct']) & (tmp['entry_ticks'] > 1)].copy()
tmp['sym'] = tmp.index.get_level_values('symbol').astype('str')
tmp = tmp.loc[~tmp['sym'].str.match('^(\+[0-9]|.).*:.*')]
sdict = {f'lsn_win_{.1 * q:.2f}': lambda x: int(dview.TimedeltaView(x).normal_microseconds()) for q in range(1, 10)}
tmp = src.sniper_opp_dur_summarizer.get_opp_dur_summary(tmp.groupby('sym'))
tmp.loc[(tmp['lsn_win_0.50'] > pd.Timedelta('0us'))]\
    .drop(columns=[f'fsn_win_{.1 * q:.2f}' for q in range(1, 10)])\
    .sort_values('n_opps', ascending=False)\
    .head(25).style.format(sdict)

,,n_opps,lsn_win_0.10,lsn_win_0.20,lsn_win_0.30,lsn_win_0.40,lsn_win_0.50,lsn_win_0.60,lsn_win_0.70,lsn_win_0.80,lsn_win_0.90
sym,,,,,,,,,,,
+(GFQ1-GFU1) -(GFQ1-GFF2) +(GFU1-GFH2) -(GFF2-GFH2),0,19,1493,2081,3077,5878,11922,19988,37860,67221,305061
+(GFQ1-GFU1) -(GFQ1-GFV1) +(GFU1-GFH2) -(GFV1-GFH2),0,17,-35,37,97,661,2063,12360,41593,300938,1037362
+(GFU1-GFF2) -(GFU1-GFH2) +(GFF2-GFH2),0,16,-130,109,913,2015,12402,25487,83406,369180,1610263
+GFU1 -GFV1 -(GFU1-GFH2) +(GFV1-GFH2),0,15,75,221,4194,11351,17686,95551,129636,282322,337914
+HEQ1 -(HEQ1-HEV1) -(HEV1-HEM2) -HEM2,0,11,-250,-160,-151,9,42,103,225,256,66718
+(GFU1-GFX1) -(GFV1-GFX1) -(GFU1-GFH2) +(GFV1-GFH2),0,11,-55,-21,109,139,634,2063,90342,133391,348889
+HEV1 -(HEV1-HEM2) -HEJ2 +(HEJ2-HEM2),0,11,-250,-160,-151,9,42,103,225,256,66718
+HEV1 -(HEV1-HEM2) -HEM2,0,11,-250,-160,-151,9,42,103,225,256,66718
+(ZMQ1-ZMZ1) -(ZMQ1-ZMN2) +(ZMZ1-ZMN2),0,10,26,45,51,68,90,103,130,206,615


In [132]:
tmp.loc[tmp['lsn_win_0.50'] > pd.Timedelta('0us')]\
    .drop(columns=[f'fsn_win_{.1 * q:.2f}' for q in range(1, 10)])\
    .sort_values('n_opps', ascending=False)\
    .head(25).style.format(sdict)

,,n_opps,lsn_win_0.10,lsn_win_0.20,lsn_win_0.30,lsn_win_0.40,lsn_win_0.50,lsn_win_0.60,lsn_win_0.70,lsn_win_0.80,lsn_win_0.90
sym,,,,,,,,,,,
+(HEQ1-HEV1) -(HEQ1-HEZ1) +(HEV1-HEZ1),0,45,-1403,-377,-125,-40,1,36,80,162,455
+(GFQ1-GFU1) -(GFQ1-GFV1) +(GFU1-GFH2) -(GFV1-GFH2),0,41,48,164,918,11355,35702,114615,281473,531217,1752161
+GFU1 -GFV1 -(GFU1-GFH2) +(GFV1-GFH2),0,39,-28,58,195,997,8831,28333,101906,207136,303117
+(GFQ1-GFU1) -(GFQ1-GFF2) +(GFU1-GFH2) -(GFF2-GFH2),0,36,111,1850,2358,5832,11340,20120,50713,99985,687571
+(GFU1-GFF2) -(GFX1-GFF2) -(GFU1-GFH2) +(GFX1-GFH2),0,26,-45,109,169,1730,40219,66214,136459,386015,2329537
+GFU1 -GFX1 -(GFU1-GFH2) +(GFX1-GFH2),0,25,305,20017,51403,149183,187295,442244,1056829,1793765,2493867
+HEJ2 -(HEJ2-HEQ2) -HEN2 +(HEN2-HEQ2),0,24,2509,22014,66936,194818,972703,1830456,6006446,8329972,12204668
+(GFU1-GFV1) -(GFU1-GFH2) +(GFV1-GFH2),0,22,40,128,378,8246,121101,288202,1438665,2415337,3531003
+HEV1 -(HEV1-HEJ2) -(HEJ2-HEN2) -HEN2,0,22,37,224,9333,85354,105505,649202,975333,2364227,3294235


In [130]:
tmp = topps
tmp = tmp.loc[(tmp['is_direct']) & (tmp['entry_ticks'] > 0)].copy()
tmp['sym'] = tmp.index.get_level_values('symbol')
tmp['sym'] = tmp['sym'].astype('str')
tmp = tmp.loc[~tmp['sym'].str.match('^(\+[0-9]|.).*:.*')]
sdict = {f'lsn_win_{.1 * q:.2f}': lambda x: int(dview.TimedeltaView(x).normal_microseconds()) for q in range(1, 10)}
tmp = src.sniper_opp_dur_summarizer.get_opp_dur_summary(tmp.groupby('sym'))
tmp.drop(columns=[f'fsn_win_{.1 * q:.2f}' for q in range(1, 10)]).sort_values('n_opps', ascending=False).head(25).style.format(sdict)

,,n_opps,lsn_win_0.10,lsn_win_0.20,lsn_win_0.30,lsn_win_0.40,lsn_win_0.50,lsn_win_0.60,lsn_win_0.70,lsn_win_0.80,lsn_win_0.90
sym,,,,,,,,,,,
+(HEQ1-HEV1) -(HEQ1-HEZ1) +(HEV1-HEZ1),0,45,-1403,-377,-125,-40,1,36,80,162,455
+(GFQ1-GFU1) -(GFQ1-GFV1) +(GFU1-GFH2) -(GFV1-GFH2),0,41,48,164,918,11355,35702,114615,281473,531217,1752161
+GFU1 -GFV1 -(GFU1-GFH2) +(GFV1-GFH2),0,39,-28,58,195,997,8831,28333,101906,207136,303117
+(GFQ1-GFU1) -(GFQ1-GFF2) +(GFU1-GFH2) -(GFF2-GFH2),0,36,111,1850,2358,5832,11340,20120,50713,99985,687571
+(ZSQ1-ZSU1) -(ZSQ1-ZSX1) +(ZSU1-ZSX1),0,33,-8422,-6621,-3451,-1890,-351,101,329,1797,4480
+(GFU1-GFF2) -(GFX1-GFF2) -(GFU1-GFH2) +(GFX1-GFH2),0,26,-45,109,169,1730,40219,66214,136459,386015,2329537
+GFU1 -GFX1 -(GFU1-GFH2) +(GFX1-GFH2),0,25,305,20017,51403,149183,187295,442244,1056829,1793765,2493867
+HEJ2 -(HEJ2-HEQ2) -HEN2 +(HEN2-HEQ2),0,24,2509,22014,66936,194818,972703,1830456,6006446,8329972,12204668
+HEV1 -(HEV1-HEJ2) -(HEJ2-HEN2) -HEN2,0,22,37,224,9333,85354,105505,649202,975333,2364227,3294235


In [81]:
sdict = {f'lsn_win_{.1 * q:.2f}': lambda x: dview.TimedeltaView(x).normal_microseconds() for q in range(1, 10)}
topps_dur_summ.drop(columns=[f'fsn_win_{.1 * q:.2f}' for q in range(1, 10)]).sort_values('n_opps', ascending=False).head(25).style.format(sdict)

,,n_opps,lsn_win_0.10,lsn_win_0.20,lsn_win_0.30,lsn_win_0.40,lsn_win_0.50,lsn_win_0.60,lsn_win_0.70,lsn_win_0.80,lsn_win_0.90
symbol,,,,,,,,,,,
+HEJ2 -(HEJ2-HEQ2) -HEN2 +(HEN2-HEQ2),0,213,58,514,4536,22014,90980,497019,207332,65276,587490
+HEJ2 -(HEJ2-HEQ2) -HEQ2,0,122,42,469,5596,30694,120966,639723,228321,727174,337603
+HEG2 -(HEG2-HEJ2) -(HEJ2-HEQ2) -HEQ2,0,94,999869,144,764,6487,75516,360101,145470,845526,361262
+(GFU1-GFX1) +GFX1 -(GFU1-GFH2) -GFH2,0,93,999873,33,66,338,2203,13129,71550,178073,868698
+(HEG2-HEJ2) -(HEG2-HEN2) +(HEJ2-HEQ2) -(HEN2-HEQ2),0,74,94,730,5722,75239,258298,748254,696344,845526,732275
+NGG2 -(NGG2-NGH2) -(NGH2-NGJ2) -NGJ2,0,73,964175,976599,982798,988387,993083,996780,999939,61,926
+HEV1 -(HEV1-HEZ1) -(HEZ1-HEG2) -HEG2,0,67,991674,998290,999650,999877,999976,6,28,34,44
+GFU1 -GFV1 -(GFU1-GFH2) +(GFV1-GFH2),0,66,999908,36,111,587,10931,25065,126885,241839,558650
+GFQ1 -(GFQ1-GFU1) -(GFU1-GFH2) -GFH2,0,64,33,186,897,7138,29309,93102,293238,671414,634823


In [79]:
sdict = {f'lsn_win_{.1 * q:.2f}': lambda x: x.microseconds for q in range(1, 10)}
topps_good_dur_summ.drop(columns=[f'fsn_win_{.1 * q:.2f}' for q in range(1, 10)]).sort_values('n_opps', ascending=False).head(25).style.format(sdict)

,,n_opps,lsn_win_0.10,lsn_win_0.20,lsn_win_0.30,lsn_win_0.40,lsn_win_0.50,lsn_win_0.60,lsn_win_0.70,lsn_win_0.80,lsn_win_0.90
symbol,,,,,,,,,,,
+HEJ2 -(HEJ2-HEQ2) -HEN2 +(HEN2-HEQ2),0,193,351,2890,11941,68539,248188,735672,850718,524263,618692
+HEJ2 -(HEJ2-HEQ2) -HEQ2,0,107,712,5574,25847,96797,535257,829321,846123,18307,376438
+HEG2 -(HEG2-HEJ2) -(HEJ2-HEQ2) -HEQ2,0,77,520,2758,11185,75990,293320,762996,846123,134735,469707
+(GFU1-GFX1) +GFX1 -(GFU1-GFH2) -GFH2,0,68,85,408,1790,11640,25715,81245,146490,371212,852181
+(HEG2-HEJ2) -(HEG2-HEN2) +(HEJ2-HEQ2) -(HEN2-HEQ2),0,67,567,4199,24012,93122,557468,945340,846123,911319,257790
+(HEJ2-HEN2) -(HEJ2-HEQ2) +(HEN2-HEQ2),0,58,1682,13072,71173,108878,572401,120186,415788,730893,574898
+(HEJ2-HEM2) -(HEJ2-HEQ2) +(HEM2-HEN2) +(HEN2-HEQ2),0,55,200,1594,7139,13713,45813,155490,809788,525707,347882
+GFQ1 -(GFQ1-GFU1) -(GFU1-GFH2) -GFH2,0,54,338,2651,10514,26833,65935,204030,462161,161148,117295
+(GFU1-GFV1) +GFV1 -(GFU1-GFH2) -GFH2,0,52,475,8645,14229,32635,72974,136540,356244,135142,907519


In [60]:
synth_secs = SyntheticSecurityLoader().result().set_index('id')
synth_secs
# market_dates = get_feather_dates(340, name = f'security', month=[7])
# opp_root = opps.copy()
# opp_root = opp_root.set_index(opp_root.index.set_names(['market_date', 'symbol', 'opp_start_eid']))
# opp = opp_root.copy()
# synth_bk_loader = src.synthetic_book_loader.SyntheticBookLoader(340, synth_secs.loc[list(range(12,25)),:], dates=market_dates)

,symbol,non_member_fee,member_fee,member106j_fee
id,,,,
1,+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),11.52,4.68,2.88
2,+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...,15.36,6.24,3.84
3,+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-...,15.36,6.24,3.84
4,+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,11.58,4.74,2.94
5,+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-...,15.36,6.24,3.84
6,+ZSQ1 -(ZSQ1-ZSU1) -ZSU1,7.74,3.18,1.98
7,+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,7.74,3.18,1.98
8,+ZSN2 -(ZSN2-ZSX2) -ZSX2,7.74,3.18,1.98
9,+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),11.52,4.68,2.88


In [55]:
synth_secs
# synth_bk_jul = synth_bk_loader.load_synth_book(symbols=[''])

,symbol,non_member_fee,member_fee,member106j_fee
id,,,,
1,+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),11.52,4.68,2.88
2,+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...,15.36,6.24,3.84
3,+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-...,15.36,6.24,3.84
4,+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,11.58,4.74,2.94
5,+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-...,15.36,6.24,3.84
6,+ZSQ1 -(ZSQ1-ZSU1) -ZSU1,7.74,3.18,1.98
7,+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,7.74,3.18,1.98
8,+ZSN2 -(ZSN2-ZSX2) -ZSX2,7.74,3.18,1.98
9,+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),11.52,4.68,2.88


In [50]:
synth_bk_date_groupby = opp_loader.synth_bk().groupby(level=[0])
# odg = opp.groupby(level=[0])
odg = opp.groupby(level='market_date')
symbols = synth_secs.loc[list(range(12, 25))]
opp_w_bk_root = SyntheticBookMerger(odg, synth_bk_date_groupby).merge(parallel=True)

NameError: name 'opp_loader' is not defined

In [8]:
opp_bk_tmp = opp_w_bk_root.copy()
opp_bk_tmp['t_tod'] = opp_bk_tmp['t_time'] - opp_bk_tmp.index.get_level_values('market_date') - pd.Timedelta('1d')
opp_bk_tmp = dview.with_groupby_origin_col(opp_bk_tmp, levels=['market_date', 'symbol', 'opp_start_eid'])
ov_tmp = to_opp_view(opp_bk_tmp)
display_opp_view(ov_tmp)

In [12]:
opp_bk= opp_w_bk_root.copy()
opp_bk = pd.merge(
    opp_bk.reset_index('symbol').rename(columns={'symbol': 'sym'}),
    synth_secs, left_on='sym', right_on='symbol', how='left')\
    .set_index(opp_bk.index)

opp_bk_edge_summ = get_edge_summary(opp_bk, cpp=50.0, fee='member106j_fee')
es = opp_bk_edge_summ.copy()
edge_col_names = [(f'{e * .25:.2f}', dview.fmt_float_lambda(ndec=0)) for e in range(-2, 5)]
sdict = dict([(c, dview.fmt_float_lambda(ndec=2)) for c in ['net', 'net_cash', 'fill_edge', 'net_fill_cash']] + edge_col_names)
es = es.reset_index().set_index(['market_date', 'symbol'])
es.style.format(sdict)

In [13]:
es['month'] = pd.DatetimeIndex(es.index.get_level_values('market_date')).month
es_syms = es.index.get_level_values('symbol')
es['sym'] = es_syms

def get_monthly_pnl(x):
    return pd.DataFrame(data={str(m): [x.loc[x.name,m]['net_fill_cash'].sum() if (x.name, m) in x.index else 0.0] for m in range(4,8)})

es[['net_fill_cash', 'month', 'sym']].groupby(['sym', 'month']).sum().groupby(level='sym').apply(get_monthly_pnl)

,,4,5,6,7
sym,,,,,
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),0,0.0,0.0,918.88,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2),0,0.0,0.0,-41.78,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),0,0.0,0.0,807.66,0.0
+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),0,0.0,0.0,-1210.74,0.0
+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2),0,0.0,0.0,-238.66,0.0
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,0,0.0,0.0,-149.90,0.0
+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,0,0.0,0.0,-193.80,0.0
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,0,0.0,0.0,-393.24,0.0
+ZSN1 -(ZSN1-ZSX1) -ZSX1,0,0.0,0.0,-226.60,0.0


In [166]:
es['month'] = pd.DatetimeIndex(es.index.get_level_values('market_date')).month
es_syms = es.index.get_level_values('symbol')
es['sym'] = es_syms

def get_monthly_pnl(x):
    return pd.DataFrame(data={str(m): [x.loc[x.name,m]['net_cash'].sum() if (x.name, m) in x.index else 0.0] for m in range(4,8)})

es[['net_cash', 'month', 'sym']].groupby(['sym', 'month']).sum().groupby(level='sym').apply(get_monthly_pnl)
# es.reset_index('symbol')
# es.groupby(level='symbol').sum()
# len(es.index.unique())

,,4,5,6,7
sym,,,,,
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),0,0.0,0.0,562.5,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2),0,0.0,0.0,550.0,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),0,0.0,0.0,2587.5,0.0
+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),0,0.0,0.0,-712.5,0.0
+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2),0,0.0,0.0,25.0,0.0
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,0,0.0,0.0,100.0,0.0
+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,0,0.0,0.0,-75.0,0.0
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,0,0.0,0.0,-37.5,0.0
+ZSN1 -(ZSN1-ZSX1) -ZSX1,0,0.0,0.0,-137.5,0.0
